In [76]:
from analysis import actions
import pandas as pd
import re

In [107]:


def regex(string):
  matcher = re.compile(string, re.IGNORECASE)
  
  return lambda x : matcher.search(x)


collected = {}


def referred(action):
  return regex("^referred")(action)


def scheduled(action): 
  return regex("scheduled")(action)


def read(action):
    # return re.compile("(read[.,;']|read second|read third)", re.IGNORECASE).findall(action)
    return re.compile("(read[.,;'\s]([second|third)])*)", re.IGNORECASE).findall(action)

def reported(action): 
    return re.compile("(reported \w+)", re.IGNORECASE).search(action)


for i in actions[:100]:
  if i.id not in collected:
    collected[i.id] = []
  if referred(i.action):
    collected[i.id].append("referred")
  if scheduled(i.action):
    collected[i.id].append("hearing scheduled")
  # reading = regex("(read first|second)")(i.action)
  if regex("engrossed")(i.action):
    collected[i.id].append("engrossed")
  if regex("enacted")(i.action):
    collected[i.id].append("enacted")
  connected = regex("([HS]\d{4})")(i.action)
  if connected: 
    collected[i.id].append(connected.group())
  readings = read(i.action)
  if readings: 
    collected[i.id].extend(readings)
  report = reported(i.action)
  if report:
    print(report.groups())
    collected[i.id].append(report.group())


display(collected)

('reported favorably',)
('reported favorably',)
('reported that',)
('reported favorably',)


{'H1': ['referred', 'H4000'],
 'H10': ['referred',
  'hearing scheduled',
  'hearing scheduled',
  'reported favorably'],
 'H100': ['referred',
  'reported favorably',
  'reported that',
  ('Read second', 'd'),
  'engrossed',
  ('Read third', 'd'),
  ('Read,', ''),
  ('read second', 'd'),
  'engrossed',
  ('Read third', 'd'),
  'enacted',
  'enacted'],
 'H1000': ['referred', 'hearing scheduled'],
 'H1001': ['referred', 'hearing scheduled'],
 'H1002': ['referred', 'hearing scheduled', 'H4784'],
 'H1003': ['referred', 'hearing scheduled'],
 'H1004': ['referred', 'hearing scheduled'],
 'H1005': ['referred', 'hearing scheduled'],
 'H1006': ['referred', 'hearing scheduled'],
 'H1007': ['referred', 'hearing scheduled', 'S2880'],
 'H1008': ['referred', 'hearing scheduled'],
 'H1009': ['referred', 'hearing scheduled', 'H4598'],
 'H101': ['referred', 'hearing scheduled'],
 'H1010': ['referred', 'hearing scheduled'],
 'H1011': ['referred', 'hearing scheduled'],
 'H1012': ['referred', 'hearing sc

In [114]:

from datetime import datetime
from dateutil.parser import parse

def getDate(action):
  return parse(action.date).strftime("%x %X")


bills = {}

for i in actions: 
  if i.id not in bills: 
    bills[i.id] = { 'id': i.id, 'history': []}
  bills[i.id]['history'].append(i)


bills_by_id = sorted(list(set([i.id for i in actions])))




In [192]:

referred = re.compile("(referred to .*)", re.IGNORECASE)
read = re.compile("(read[.,;'\s]([second|third)])*)", re.IGNORECASE)
rules_suspended = re.compile("(rules suspended)", re.IGNORECASE)
committee = re.compile("committee on (.*)", re.IGNORECASE)

def get_timeperiod(bill_history): 
  item = bill_history['history'][0]
  time_lag = None
  rf = None
  rd = None
  
  for h in bill_history['history']:
    if referred.search(h.action):
      fr = h
      # break
  for h in bill_history['history']:
    if read.search(h.action):
      rd = h
      break
      
  return time_lag
  


def regexer(string_to_regex):
  return re.compile(string_to_regex, re.IGNORECASE)



engrossed = regexer("(engrossed)")
new_draft = regexer("(new draft of .*)")



statuses = {
  "id": [],
  "branch": [],
  "referred": [], 
  "read": [],
  "new_draft": [],
  "rules_suspended": [], 
  "engrossed": []
}



def get_status(bill_history):
  refd = [h if referred.search(h) is not None else None for h in bill_history]
  return refd


for i,v in bills.items():
  for h in v['history']:
    statuses['id'].append(h.id)
    statuses['branch'].append(h.branch if h.branch else "-")
    ref = referred.search(h.action)
    statuses['referred'].append(ref.group() if ref else "-")
    rd = read.search(h.action)
    statuses['read'].append(rd.group() if rd else "-")
    rs = rules_suspended.search(h.action)
    statuses['rules_suspended'].append(rs.group() if rs else "-")
    eng = engrossed.search(h.action)
    statuses['engrossed'].append(eng.group() if eng else "-")
    drft = new_draft.search(h.action)
    statuses['new_draft'].append(drft.group() if drft else "-")

df = pd.DataFrame(statuses)



df.loc[df['new_draft'] != "-"]


,id,branch,referred,read,new_draft,rules_suspended,engrossed
105,H102,House,-,-,new draft of H96,-,-
14642,H3701,House,-,-,new draft of H96,-,-
14643,H3701,House,-,-,New draft of H96,-,-
15286,H3769,House,-,-,New draft of H3704,-,-
15564,H3811,House,-,-,New draft of H380,-,-
...,...,...,...,...,...,...,...
41177,S3059,Senate,-,-,New draft of S114,-,-
41319,S3078,Senate,-,-,New draft of S3060,-,-
41344,S3081,Senate,-,-,New draft of S3066,-,-
41355,S3083,Senate,-,-,New draft of S3067,-,-
